### API Router



### Prompt Template

User input does not need to be passed directly into the LLM. The prompt template provides additional context on the specific task at hand.

Prompt Templates are designed for single-turn interactions. It typically accepts a single template string with placeholders that can be dynamically filled with inputs.

```python
from langchain.prompts import PromptTemplate

template = "Translate the following English text to French: {text}"
prompt = PromptTemplate(input_variables=["text"], template=template)

filled_prompt = prompt.format(text="Hello, how are you?")
print(filled_prompt)
# Output: Translate the following English text to French: Hello, how are you?
```

### Chat Prompt Template

ChatPromptTemplate is used for multi-turn interactions or prompts designed for chat-based language models. It is composed of messages like SystemMessage, HumanMessage, and AIMessage. These represent the context, user input, and AI responses, respectively.

ChatPromptTemplate can take messages in various formats, including predefined message classes (e.g., SystemMessage, HumanMessage) as well as tuple formats. LangChain provides message classes such as SystemMessage, HumanMessage, AIMessage, and ChatMessage. These are structured representations for different roles in a chat.

```python
from langchain.prompts import ChatPromptTemplate, SystemMessage, HumanMessage

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Can you help me with my math homework?")
])
formatted_messages = chat_prompt.format_messages()
for msg in formatted_messages:
    print(f"{msg.type}: {msg.content}")
```

Instead of using the predefined message classes, you can represent messages as tuples, where the first element is the role (e.g., "system", "human", "assistant"), and the second element is the content of the message.

```python
from langchain.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "Can you help me solve a quadratic equation?"),
    ("assistant", "Of course! What's the equation?")
])
formatted_messages = chat_prompt.format_messages()
for msg in formatted_messages:
    print(f"{msg.type}: {msg.content}")
```

### Chat Prompt Template and Message Placeholder

The MessagesPlaceholder in LangChain is a utility used within ChatPromptTemplate to insert dynamic chat histories or message sequences into a prompt. This is particularly useful when you want to incorporate conversation history dynamically without hardcoding all previous messages into the prompt.

MessagesPlaceholder is commonly used when:
- Maintaining Context: You want to include past messages in a chat.
- Dynamic History: The history may change or be trimmed based on application logic.
- Trimming: You can preprocess the chat_history (e.g., trim irrelevant messages) before passing it to the prompt.

Here’s how to use MessagesPlaceholder in a ChatPromptTemplate:

```python
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("What do you think about {topic}?")
])
```

At runtime, you pass the actual chat history to replace the MessagesPlaceholder:

```python
from langchain.schema import AIMessage, HumanMessage

chat_history = [
    HumanMessage(content="Can you summarize quantum physics?"),
    AIMessage(content="Sure! Quantum physics deals with subatomic particles and wave-particle duality."),
]

formatted_messages = chat_prompt.format_messages(
    chat_history=chat_history,  # Replace placeholder
    topic="black holes"
)

for message in formatted_messages:
    print(f"{message.type}: {message.content}")
```

### Chat Prompt Template and LCEL

PromptTemplate and ChatPromptTemplate implement the Runnable interface, the basic building block of the LangChain Expression Language (LCEL). This means they support invoke, ainvoke, stream, astream, batch, abatch, astream_log calls.

PromptTemplate accepts a dictionary (of the prompt variables) and returns a StringPromptValue. A ChatPromptTemplate accepts a dictionary and returns a ChatPromptValue.

```python
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

prompt_val = prompt_template.invoke({"adjective": "funny", "content": "chickens"})
prompt_val # StringPromptValue(text='Tell me a funny joke about chickens.')
prompt_val.to_string() # 'Tell me a funny joke about chickens.'
prompt_val.to_messages() # [HumanMessage(content='Tell me a funny joke about chickens.')]

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
chat_val = chat_template.invoke({"text": "i dont like eating tasty things."})
chat_val.to_messages()
# [SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."),
```

### Few Shot Prompt Templates and Example Selectors

The purpose of Example Selectors in LangChain is to dynamically choose the most relevant examples to include in a prompt. This is especially important when you have a large dataset of examples and cannot include all of them due to constraints like token limits or relevance to the current input. Example selectors ensure that the chosen examples are tailored to the specific input, improving the quality of responses from the language model.

Large language models like GPT often perform better with few-shot learning, where the prompt includes input-output examples to demonstrate the task. Including irrelevant or too many examples can lead to suboptimal responses or exceed token limits.

Example Selectors dynamically choose the most relevant subset of examples based on the current input, ensuring optimal context is provided to the model.

LangChain provides several prebuilt example selectors, and you can also implement custom ones. Common types include:
- Similarity Selector
    - huih
- Max Marginal Relevance (MMR) Selector
    - Balances relevance and diversity by selecting examples that are similar to the input but not too similar to each other.
- Length-Based Selector
    - Chooses examples that fit within a token limit or are of a similar length to the input.
- N-gram Overlap Selector
    - Selects examples based on shared n-grams between the input and examples.

Select by length example selector selects which examples to use based on length. This is useful when you are worried about constructing a prompt that will go over the length of the context window. For longer inputs, it will select fewer examples to include, while for shorter inputs it will select more.

```python
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

example_selector = LengthBasedExampleSelector(
    # The examples it has available to choose from.
    examples=examples,
    # The PromptTemplate being used to format the examples.
    example_prompt=example_prompt,
    # The maximum length that the formatted examples should be.
    # Length is measured by the get_text_length function below.
    max_length=25,
    # The function used to get the length of a string, which is used
    # to determine which examples to include. It is commented out because
    # it is provided as a default value if none is specified.
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)
```

The MaxMarginalRelevanceExampleSelector selects examples based on a combination of which examples are most similar to the inputs, while also optimizing for diversity. It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs, and then iteratively adding them while penalizing them for closeness to already selected examples.

```python
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=2,
)

mmr_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

# Input is a feeling, so should select the happy/sad example as the first one
print(mmr_prompt.format(adjective="worried"))
Give the antonym of every input

Input: happy
Output: sad

Input: windy
Output: calm

Input: worried
Output:


# Let's compare this to what we would just get if we went solely off of similarity,
# by using SemanticSimilarityExampleSelector instead of MaxMarginalRelevanceExampleSelector.
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=2,
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)
print(similar_prompt.format(adjective="worried"))
Give the antonym of every input

Input: happy
Output: sad

Input: sunny
Output: gloomy

Input: worried
Output:
```

The NGramOverlapExampleSelector selects and orders examples based on which examples are most similar to the input, according to an ngram overlap score. The ngram overlap score is a float between 0.0 and 1.0, inclusive.

The selector allows for a threshold score to be set. Examples with an ngram overlap score less than or equal to the threshold are excluded. The threshold is set to -1.0, by default, so will not exclude any examples, only reorder them. Setting the threshold to 0.0 will exclude examples that have no ngram overlaps with the input.

```python
from langchain_community.example_selector.ngram_overlap import (
    NGramOverlapExampleSelector,
)
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Examples of a fictional translation task.
examples = [
    {"input": "See Spot run.", "output": "Ver correr a Spot."},
    {"input": "My dog barks.", "output": "Mi perro ladra."},
    {"input": "Spot can run.", "output": "Spot puede correr."},
]

example_selector = NGramOverlapExampleSelector(
    # The examples it has available to choose from.
    examples=examples,
    # The PromptTemplate being used to format the examples.
    example_prompt=example_prompt,
    # The threshold, at which selector stops.
    # It is set to -1.0 by default.
    threshold=-1.0,
    # For negative threshold:
    # Selector sorts examples by ngram overlap score, and excludes none.
    # For threshold greater than 1.0:
    # Selector excludes all examples, and returns an empty list.
    # For threshold equal to 0.0:
    # Selector sorts examples by ngram overlap score,
    # and excludes those with no ngram overlap with input.
)
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the Spanish translation of every input",
    suffix="Input: {sentence}\nOutput:",
    input_variables=["sentence"],
)

# An example input with large ngram overlap with "Spot can run."
# and no overlap with "My dog barks."
print(dynamic_prompt.format(sentence="Spot can run fast."))
Give the Spanish translation of every input

Input: Spot can run.
Output: Spot puede correr.

Input: See Spot run.
Output: Ver correr a Spot.

Input: My dog barks.
Output: Mi perro ladra.

Input: Spot can run fast.
Output:

Select by similarity selects examples based on similarity to the inputs. It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs.

```python
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # The number of examples to produce.
    k=1,
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(adjective="worried"))
Give the antonym of every input

Input: happy
Output: sad

Input: worried
Output:
```

### Few Shot Chat Message Prompt Template

FewShotPromptTemplate is a general-purpose template for few-shot prompting. Examples are stored as plain text templates or dictionaries. Examples are formatted using a simple PromptTemplate. Includes a prefix (instructions or context) and a suffix (the current input or question). Dynamically selects the most relevant examples using an ExampleSelector (e.g., SemanticSimilarityExampleSelector).

FewShotChatMessagePromptTemplate is specifically designed for chat-based models (e.g., ChatGPT, Anthropic, or any model that uses chat messages). Formats examples into chat-style messages, distinguishing between system, human, and ai roles. Each example is converted into a sequence of chat messages (e.g., Human: Input\nAI: Output). Uses ChatPromptTemplate to define message roles. Can dynamically insert examples in a conversational format using ExampleSelector. Suitable for conversational tasks where examples need to be formatted as chat exchanges.

```python
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "3+3", "output": "6"},
]

few_shot_chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_chat_prompt.format())
Human: 2+2
AI: 4
Human: 3+3
AI: 6

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_chat_prompt,
        ("human", "{input}"),
    ]
)

from langchain_community.chat_models import ChatAnthropic

chain = final_prompt | ChatAnthropic(temperature=0.0)

chain.invoke({"input": "What's the square of a triangle?"})
AIMessage(content=' Triangles do not have a "square". A square refers to a shape with 4 equal sides and 4 right angles. Triangles have 3 sides and 3 angles.\n\nThe area of a triangle can be calculated using the formula:\n\nA = 1/2 * b * h\n\nWhere:\n\nA is the area \nb is the base (the length of one of the sides)\nh is the height (the length from the base to the opposite vertex)\n\nSo the area depends on the specific dimensions of the triangle. There is no single "square of a triangle". The area can vary greatly depending on the base and height measurements.', additional_kwargs={}, example=False)
```

Sometimes you may want to condition which examples are shown based on the input. For this, you can replace the examples with an example_selector.

```python
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "horse"})

from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

from langchain_community.chat_models import ChatAnthropic

chain = final_prompt | ChatAnthropic(temperature=0.0)

chain.invoke({"input": "What's 3+3?"})
# AIMessage(content=' 3 + 3 = 6', additional_kwargs={}, example=False)
```

### Partial Prompt Templates

It can make sense to "partial" a prompt template - e.g. pass in a subset of the required values, as to create a new prompt template which expects only the remaining subset of values.

One common use case for wanting to partial a prompt template is if you get some of the variables before others. For example, suppose you have a prompt template that requires two variables, foo and baz. If you get the foo value early on in the chain, but the baz value later, it can be annoying to wait until you have both variables in the same place to pass them to the prompt template. Instead, you can partial the prompt template with the foo value, and then pass the partialed prompt template along and just use that. Below is an example of doing this:

```python
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{foo}{bar}")
partial_prompt = prompt.partial(foo="foo")
print(partial_prompt.format(bar="baz"))
```

The other common use is to partial with a function. The use case for this is when you have a variable you know that you always want to fetch in a common way. A prime example of this is with date or time. Imagine you have a prompt which you always want to have the current date. You can't hard code it in the prompt, and passing it along with the other input variables is a bit annoying. In this case, it's very handy to be able to partial the prompt with a function that always returns the current date.

```python
from datetime import datetime


def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))
```

You can also just initialize the prompt with the partialed variables, which often makes more sense in this workflow.

```python
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective"],
    partial_variables={"date": _get_datetime},
)
print(prompt.format(adjective="funny"))
```

### Prompt Composition

LangChain provides a user friendly interface for composing different parts of prompts together. You can do this with either string prompts or chat prompts. Constructing prompts this way allows for easy reuse.

When working with string prompts, each template is joined together. You can work with either prompts directly or strings:

```python
from langchain_core.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)
prompt
# PromptTemplate(input_variables=['language', 'topic'], template='Tell me a joke about {topic}, make it funny\n\nand in {language}')
```

LangChain includes an abstraction PipelinePromptTemplate, which can be useful when you want to reuse parts of prompts. 

```python
from langchain_core.prompts.pipeline import PipelinePromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

pipeline_prompt.input_variables # ['example_q', 'person', 'input', 'example_a']

print(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)
You are impersonating Elon Musk.

Here's an example of an interaction:

Q: What's your favorite car?
A: Tesla

Now, do this for real!

Q: What's your favorite social media site?
A:
```


### LLM

The LLM is a pure text completion model. The LLM takes a string prompt as input and output a string completion.

### Chat Model

Chat Models are built on top of LLMs. The LLM objects take string as input and output string. The ChatModel objects take a list of messages as input and output a message. While chat models use language models under the hood, the interface they use is a bit different. Rather than using a "text in, text out" API, they use an interface where "chat messages" are the inputs and outputs.

A chat model is a language model that uses chat messages as inputs and returns chat messages as outputs (as opposed to using plain text). LangChain has integrations with many model providers (OpenAI, Cohere, Hugging Face, etc.) and exposes a standard interface to interact with all of these models.

The chat model interface is based around messages rather than raw text. The types of messages currently supported in LangChain are AIMessage, HumanMessage, SystemMessage, FunctionMessage and ChatMessage -- ChatMessage takes in an arbitrary role parameter. Most of the time, you'll just be dealing with HumanMessage, AIMessage, and SystemMessage.

### Chat Model and LCEL

Chat models implement the Runnable interface, the basic building block of the LangChain Expression Language (LCEL). This means they support invoke, ainvoke, stream, astream, batch, abatch, astream_log calls.

Chat models accept List[BaseMessage] as inputs, or objects which can be coerced to messages, including str (converted to HumanMessage) and PromptValue.

```python
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="What is the purpose of model regularization?"),
]

# using invoke
chat.invoke(messages)
# AIMessage(content="The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model becomes too complex and starts to fit the noise in the training data, leading to poor generalization on unseen data. Regularization techniques introduce additional constraints or penalties to the model's objective function, discouraging it from becoming overly complex and promoting simpler and more generalizable models. Regularization helps to strike a balance between fitting the training data well and avoiding overfitting, leading to better performance on new, unseen data.")

# using stream
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)
# The purpose of model regularization is to prevent overfitting and improve the generalization of a machine learning model. Overfitting occurs when a model is too complex and learns the noise or random variations in the training data, which leads to poor performance on new, unseen data. Regularization techniques introduce additional constraints or penalties to the model's learning process, discouraging it from fitting the noise and reducing the complexity of the model. This helps to improve the model's ability to generalize well and make accurate predictions on unseen data.    

# using batch
chat.batch([messages])
# [AIMessage(content="The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model becomes too complex and starts to learn the noise or random fluctuations in the training data, rather than the underlying patterns or relationships. Regularization techniques add a penalty term to the model's objective function, which discourages the model from becoming too complex and helps it generalize better to new, unseen data. This improves the model's ability to make accurate predictions on new data by reducing the variance and increasing the model's overall performance.")]

# using ainvoke
await chat.ainvoke(messages)
# AIMessage(content='The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model becomes too complex and starts to memorize the training data instead of learning general patterns and relationships. This leads to poor performance on new, unseen data.\n\nRegularization techniques introduce additional constraints or penalties to the model during training, discouraging it from becoming overly complex. This helps to strike a balance between fitting the training data well and generalizing to new data. Regularization techniques can include adding a penalty term to the loss function, such as L1 or L2 regularization, or using techniques like dropout or early stopping. By regularizing the model, it encourages it to learn the most relevant features and reduce the impact of noise or outliers in the data.')

# using astream
async for chunk in chat.astream(messages):
    print(chunk.content, end="", flush=True)
# The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model becomes too complex and starts to memorize the training data instead of learning the underlying patterns. Regularization techniques help in reducing the complexity of the model by adding a penalty to the loss function. This penalty encourages the model to have smaller weights or fewer features, making it more generalized and less prone to overfitting. The goal is to find the right balance between fitting the training data well and being able to generalize well to unseen data.

# using astream log
async for chunk in chat.astream_log(messages):
    print(chunk) 
```

### Chat Models and Langsmith

All ChatModels come with built-in LangSmith tracing. Just set the following environment variables:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY=<your-api-key>
```

and any ChatModel invocation (whether it's nested in a chain or not) will automatically be traced. A trace will include inputs, outputs, latency, token usage, invocation params, environment params, and more.

In LangSmith you can then provide feedback for any trace, compile annotated datasets for evals, debug performance in the playground, and more.

### Chat Model Message types

ChatModels take a list of messages as input and return a message. There are a few different types of messages. All messages have a role and a content property. The role describes WHO is saying the message. LangChain has different message classes for different roles. The content property describes the content of the message. This can be a few different things:
- A string (most models deal this type of content)
- A List of dictionaries (this is used for multi-modal input, where the dictionary contains information about that input type and that input location)

In addition, messages have an additional_kwargs property. This is where additional information about messages can be passed. This is largely used for input parameters that are provider specific and not general. The best known example of this is function_call from OpenAI.

Message Types:
- HumanMessage
    - This represents a message from the user. Generally consists only of content.
- AIMessage
    - This represents a message from the model. This may have additional_kwargs in it - for example tool_calls if using OpenAI tool calling.
- SystemMessage
    - This represents a system message, which tells the model how to behave. This generally only consists of content. Not every model supports this.
- FunctionMessage
    - This represents the result of a function call. In addition to role and content, this message has a name parameter which conveys the name of the function that was called to produce this result.
- ToolMessage
    - This represents the result of a tool call. This is distinct from a FunctionMessage in order to match OpenAI's function and tool message types. In addition to role and content, this message has a tool_call_id parameter which conveys the id of the call to the tool that was called to produce this result.

### Chat Models and Streaming

All ChatModels implement the Runnable interface, which comes with default implementations of all methods, ie. ainvoke, batch, abatch, stream, astream. This gives all ChatModels basic support for streaming.

Streaming support defaults to returning an Iterator (or AsyncIterator in the case of async streaming) of a single value, the final result returned by the underlying ChatModel provider. This obviously doesn't give you token-by-token streaming, which requires native support from the ChatModel provider, but ensures your code that expects an iterator of tokens can work for any of our ChatModel integrations.

### Chat Models and Tool calling

Tool Calling in LangChain is a feature that allows a language model to generate structured output by simulating the "calling" of a predefined tool. This process involves the model creating arguments that match a user-defined schema for a tool, but the actual execution of the tool (or even whether the tool is executed) is left to the user.

When we define a tool in LangChain, we specify:
- What the tool does: Its purpose or functionality.
- What inputs (arguments) it requires: A schema that defines the parameters the tool needs to execute.

The model "creates arguments" by generating these input values based on the user's query or prompt. Consider a tool for basic arithmetic operations:

```python
class CalculatorTool(BaseTool):
    name = "calculate"
    args_schema = {
        "operation": {"type": "string", "enum": ["add", "subtract", "multiply", "divide"]},
        "num1": {"type": "number"},
        "num2": {"type": "number"},
    }
```
- Operation: Specifies the type of calculation (e.g., "add").
- The first number for the operation (e.g., 5).
- The second number for the operation (e.g., 7).

When a user provides input like: "Add 5 and 7." The model interprets the query and generates the following arguments:

```python
{
    "operation": "add",
    "num1": 5,
    "num2": 7
}
```
These arguments match the schema defined in the example CalculatorTool.

In effect, despite the name, tool calling does not involve the model directly performing actions. Instead, the model provides parameters or arguments that conform to the schema, which can then be used by the user to trigger the tool or extract structured data.

How Tool Calling Works:
- Define the Tool:
    - A "tool" in LangChain represents an action the model can "call" by generating arguments that match a predefined schema.
    - The schema specifies the inputs the tool accepts and can include validations or constraints.
- Prompt the Model:
    - The user provides a prompt or query to the model.
    - The model generates output that matches the schema of the tool, simulating a "call" to the tool.
- User Handles the Execution:
    - The user decides whether to actually execute the tool based on the generated arguments.
    - Alternatively, the user may treat the structured output directly as the final result.

Many LLM providers, including Anthropic, Cohere, Google, Mistral, OpenAI, and others, support variants of a tool calling feature. These features typically allow requests to the LLM to include available tools and their schemas, and for responses to include calls to these tools. For instance, given a search engine tool, an LLM might handle a query by first issuing a call to the search engine. The system calling the LLM can receive the tool call, execute it, and return the output to the LLM to inform its response. LangChain includes a suite of built-in tools and supports several methods for defining your own custom tools. Tool-calling is extremely useful for building tool-using chains and agents, and for getting structured outputs from models more generally.

Providers adopt different conventions for formatting tool schemas and tool calls. For instance, Anthropic returns tool calls as parsed structures within a larger content block:
```json
[
  {
    "text": "<thinking>\nI should use a tool.\n</thinking>",
    "type": "text"
  },
  {
    "id": "id_value",
    "input": {"arg_name": "arg_value"},
    "name": "tool_name",
    "type": "tool_use"
  }
]
```

whereas OpenAI separates tool calls into a distinct parameter, with arguments as JSON strings:
```json
{
  "tool_calls": [
    {
      "id": "id_value",
      "function": {
        "arguments": '{"arg_name": "arg_value"}',
        "name": "tool_name"
      },
      "type": "function"
    }
  ]
}
```

LangChain implements standard interfaces for defining tools, passing them to LLMs, and representing tool calls.

For a model to be able to invoke tools, you need to pass tool schemas to it when making a chat request. LangChain ChatModels supporting tool calling features implement a .bind_tools method, which receives a list of LangChain tool objects, Pydantic classes, or JSON Schemas and binds them to the chat model in the provider-specific expected format. That's a mouthful. Let's dissect this:
- A tool in LangChain represents a predefined action the model can "invoke" by generating structured arguments that match a schema.
- Binding Tools to the Model
    - Tools must be bound to the model so that the model knows which tools are available and can generate arguments for them.
    - Binding tools involves passing the tool schemas to the model, which adapts them to the specific format expected by the provider’s API (e.g., OpenAI, Anthropic).
- .bind_tools() Method
    - The .bind_tools() method in LangChain allows you to attach tools to a ChatModel instance.
    - Once tools are bound, every subsequent API call to the model will include the tool schemas, enabling the model to simulate invoking them.

Example:

```python
from langchain.tools import BaseTool

class CalculatorTool(BaseTool):
    name = "calculate"
    description = "Performs arithmetic operations"
    args_schema = {
        "operation": {"type": "string", "enum": ["add", "subtract", "multiply", "divide"]},
        "num1": {"type": "number"},
        "num2": {"type": "number"},
    }

    def _run(self, operation, num1, num2):
        if operation == "add":
            return num1 + num2
        elif operation == "subtract":
            return num1 - num2
        elif operation == "multiply":
            return num1 * num2
        elif operation == "divide":
            return num1 / num2
        else:
            raise ValueError("Invalid operation")

# Use .bind_tools() to attach the tools to the model.
from langchain.chat_models import ChatOpenAI

# Instantiate the model
llm = ChatOpenAI(temperature=0)

# Instantiate the tool
calculator_tool = CalculatorTool()

# Bind the tool to the model
llm_with_tools = llm.bind_tools([calculator_tool])
```

How Binding Works:
- Schema Passing:
    - When you call .bind_tools(), LangChain converts the tool definitions into the specific JSON schema format expected by the provider (e.g., OpenAI or Anthropic).
- Model Awareness:
    - After binding, the model becomes "aware" of the tools and their schemas. Every subsequent chat request automatically includes the tool schemas as part of the API call.
- Tool Invocation by the Model:
    - During a conversation, the model can decide whether to "call" a tool by generating arguments that match the tool schema.

When you interact with the bound model, the model can now include tool invocations in its responses.

```python
response = llm_with_tools.invoke({
    "input": "Can you add 7 and 3?"
})

print(response)

# The model generates an output in the format:
{
    "tool": "calculate",
    "arguments": {
        "operation": "add",
        "num1": 7,
        "num2": 3
    }
}
```

We can define the schema for custom tools using the @tool decorator on Python functions:
```python
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]
```

We can equivalently define the schema using Pydantic. Pydantic is useful when your tool inputs are more complex:

```python
from langchain_core.pydantic_v1 import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [add, multiply]

# We can bind them to chat models as follows:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# We can use the bind_tools() method to handle converting Multiply to a "tool" and binding it to the model (i.e., passing it in each time the model is invoked).
llm_with_tools = llm.bind_tools(tools)
```

When you just use bind_tools(tools), the model can choose whether to return one tool call, multiple tool calls, or no tool calls at all. Some models support a tool_choice parameter that gives you some ability to force the model to call a tool. For models that support this, you can pass in the name of the tool you want the model to always call tool_choice="xyz_tool_name". Or you can pass in tool_choice="any" to force the model to call at least one tool, without specifying which tool specifically.

Currently tool_choice="any" functionality is supported by OpenAI, MistralAI, FireworksAI, and Groq. Currently Anthropic does not support tool_choice at all.

If we wanted our model to always call the multiply tool we could do:
```python
always_multiply_llm = llm.bind_tools([multiply], tool_choice="multiply")
```

And if we wanted it to always call at least one of add or multiply, we could do:
```python
always_call_tool_llm = llm.bind_tools([add, multiply], tool_choice="any")
```

If tool calls are included in a LLM response, they are attached to the corresponding AIMessage or AIMessageChunk (when streaming) as a list of ToolCall objects in the .tool_calls attribute. A ToolCall is a typed dict that includes a tool name, dict of argument values, and (optionally) an identifier. Messages with no tool calls default to an empty list for this attribute.

Example:
```python
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls
# [{'name': 'multiply',
#   'args': {'a': 3, 'b': 12},
#   'id': 'call_UL7E2232GfDHIQGOM4gJfEDD'},
#  {'name': 'add',
#   'args': {'a': 11, 'b': 49},
#   'id': 'call_VKw8t5tpAuzvbHgdAXe9mjUx'}]
```

The .tool_calls attribute should contain valid tool calls. Note that on occasion, model providers may output malformed tool calls (e.g., arguments that are not valid JSON). When parsing fails in these cases, instances of InvalidToolCall are populated in the .invalid_tool_calls attribute. An InvalidToolCall can have a name, string arguments, identifier, and error message.

If desired, output parsers can further process the output. For example, we can convert back to the original Pydantic class:
```python
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[multiply, add])
chain.invoke(query)
# [multiply(a=3, b=12), add(a=11, b=49)]
```

When tools are called in a streaming context, message chunks will be populated with tool call chunk objects in a list via the .tool_call_chunks attribute.

### Output Parsers

OutputParsers convert the raw output of a language model into a format that can be used downstream. There are a few main types of OutputParsers, including:
- Convert text from LLM into structured information (e.g. JSON)
- Convert a ChatMessage into just a string
- Convert the extra information returned from a call besides the message (like OpenAI function invocation) into a string.



### LCEL and Runnable Interface